## Solar

Project Solar is an attempt to equire information about sourounding envrionment in a living space, analize it, and build predictions in order to answer:

> Determine value of artificial light to counteract its natural deficit

+ Given time of the day, provide an answer about the level of light
+ Given time of the day (and day of the week) - provide an answer if light should be on

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_pickle('data/sensing_numeric.pkl')
df.columns

Index(['motion', 'light_scaled', 'sound_scaled', 'location_black',
       'location_blue', 'location_green', 'location_orange', 'location_purple',
       'sun_evening', 'sun_morning', 'sun_night', 'sun_noon', 'sun_sunrise',
       'sun_sunset', 'dot_week', 'sound_log_scaled', 'light_log_scaled'],
      dtype='object')

#### Adding classification feature (predictable benchmark)

In [3]:
df['light_log_scaled'].describe()

count    373632.000000
mean          3.119393
std           2.613058
min           0.000000
25%           0.803937
50%           1.984868
75%           6.535657
max           9.339017
Name: light_log_scaled, dtype: float64

Calculate 25 percentile of light in each location.

In [4]:
def get_daily_light_percentile(dataframe, q):
    light_log_scaled_percentile = lambda x: np.percentile(x, q = q)
    
    light_daily_banchmark = dataframe.groupby([
        dataframe.index.get_level_values('timestamp').day,
        'location_black', 
        'location_blue', 
        'location_green', 
        'location_orange', 
        'location_purple']).agg({
            'light_log_scaled': light_log_scaled_percentile
        }).rename(columns={'light_log_scaled': 'light_log_scaled_percentile_{}'.format(q)})
    return light_daily_banchmark

daily_light_percentile_25 = get_daily_light_percentile(df, 25)

Add percentile to data set.

In [5]:
def add_quantile(dataframe):
    location_indexer = [
        'location_black', 
        'location_blue', 
        'location_green', 
        'location_orange', 
        'location_purple'
    ]

    for time in dataframe.index.get_level_values('timestamp').unique():
        day_group = dataframe.xs(time, level='timestamp', axis=0)
        for location_name in location_indexer:
            percentile = day_group.xs(1, level=location_name)['light_log_scaled_percentile_25']
            df.loc[((df.index.day == time) & (df[location_name] == 1)), 'daily_light_percentile_25'] = percentile.values[0] if len(percentile.values) else 0
        
add_quantile(daily_light_percentile_25)

In [6]:
motion_hourly_banchmark = df.groupby([
    df.index.get_level_values('timestamp').hour,
    'dot_week',
    'location_black', 
    'location_blue', 
    'location_green', 
    'location_orange', 
    'location_purple']).agg({'motion': ['sum', 'std']})
#motion_hourly_banchmark
#std-deviation 0.15

In [7]:
def add_motion_std(dataframe):
    location_indexer = [
        'location_black', 
        'location_blue', 
        'location_green', 
        'location_orange', 
        'location_purple'
    ]

    for time in dataframe.index.get_level_values('timestamp').unique():
        day_hour_group = dataframe.xs(time, level='timestamp', axis=0)
        for dotw in day_hour_group.index.get_level_values('dot_week').unique():
            day_group = day_hour_group.xs(dotw, level='dot_week', axis=0)
            for location_name in location_indexer:
                std_dev = day_group.xs(1, level=location_name)['motion']['std']
                df.loc[((df.index.day == time) & (df['dot_week'] == dotw) & (df[location_name] == 1)), 'daily_motion_std_dev'] = std_dev.values[0] if len(std_dev.values) else 0
        
add_motion_std(motion_hourly_banchmark)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 373632 entries, 2018-05-01 01:09:00 to 2018-05-11 23:29:00
Data columns (total 19 columns):
motion                       373632 non-null int64
light_scaled                 373632 non-null float64
sound_scaled                 373632 non-null float64
location_black               373632 non-null uint8
location_blue                373632 non-null uint8
location_green               373632 non-null uint8
location_orange              373632 non-null uint8
location_purple              373632 non-null uint8
sun_evening                  373632 non-null uint8
sun_morning                  373632 non-null uint8
sun_night                    373632 non-null uint8
sun_noon                     373632 non-null uint8
sun_sunrise                  373632 non-null uint8
sun_sunset                   373632 non-null uint8
dot_week                     373632 non-null int64
sound_log_scaled             373632 non-null float64
light_log_scaled             3736

In [9]:
MINIMUM_DAILY_HOUR_MOTION_DEVIATION = .15

def get_light_expected(row):
    return 1 if ((row['daily_motion_std_dev'] > MINIMUM_DAILY_HOUR_MOTION_DEVIATION) &
        (row['light_log_scaled'] < row['daily_light_percentile_25'])) else 0

df['light_expected'] = df.apply(get_light_expected, axis=1)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 373632 entries, 2018-05-01 01:09:00 to 2018-05-11 23:29:00
Data columns (total 20 columns):
motion                       373632 non-null int64
light_scaled                 373632 non-null float64
sound_scaled                 373632 non-null float64
location_black               373632 non-null uint8
location_blue                373632 non-null uint8
location_green               373632 non-null uint8
location_orange              373632 non-null uint8
location_purple              373632 non-null uint8
sun_evening                  373632 non-null uint8
sun_morning                  373632 non-null uint8
sun_night                    373632 non-null uint8
sun_noon                     373632 non-null uint8
sun_sunrise                  373632 non-null uint8
sun_sunset                   373632 non-null uint8
dot_week                     373632 non-null int64
sound_log_scaled             373632 non-null float64
light_log_scaled             3736

#### Feature selection (including sensing)

In [11]:
y = df['light_expected']
X = df[[
    'location_black', 
    'location_blue', 
    'location_green', 
    'location_orange', 
    'location_purple', 
    'sun_evening', 
    'sun_morning', 
    'sun_night', 
    'sun_noon', 
    'sun_sunrise', 
    'sun_sunset',
    'dot_week']]

#### Train split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

#### Logistic Regression

In [18]:
from sklearn.linear_model import LogisticRegression
from helpers import print_predict_scores
logReg = LogisticRegression()

In [19]:
logReg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [20]:
prediction = logReg.predict(X_train)
print_predict_scores(y_train, prediction)

mean_squared_error:
0.12764074454721935

precision_score:
0.5846759224886777

recall_score:
0.20696995978869354

classification_report:
             precision    recall  f1-score   support

          0       0.89      0.98      0.93    242175
          1       0.58      0.21      0.31     38049

avg / total       0.85      0.87      0.84    280224


mean square root: 0.35726844885494624



In [21]:
print('Train score r2:', logReg.score(X_train,y_train))
print('Test score r2:', logReg.score(X_test,y_test))

Train score r2: 0.872359255453
Test score r2: 0.872869561494


#### Stochastic Gradient Descent classifier

In [22]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier()

In [23]:
sgd.fit(X_train, y_train)

/Users/radek.czarnecki/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [24]:
prediction = sgd.predict(X_train)
print_predict_scores(y_train, prediction)

mean_squared_error:
0.13578066118533744

precision_score:
0.0

recall_score:
0.0

classification_report:
             precision    recall  f1-score   support

          0       0.86      1.00      0.93    242175
          1       0.00      0.00      0.00     38049

avg / total       0.75      0.86      0.80    280224


mean square root: 0.3684842753569512



/Users/radek.czarnecki/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/radek.czarnecki/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [25]:
print('Train score r2:', sgd.score(X_train,y_train))
print('Test score r2:', sgd.score(X_test,y_test))

Train score r2: 0.864219338815
Test score r2: 0.864668979102


#### Parameters optimization - cross validation

In [26]:
from sklearn.model_selection import GridSearchCV, KFold

In [27]:
param_grid = [
    {'C': [10**-i for i in range(-5, 5)], 'class_weight': [None, 'balanced']}
]
grid = GridSearchCV(
    estimator=logReg,
    param_grid=param_grid,
    cv=7,
    scoring = 'neg_mean_squared_error'
)

In [28]:
grid.fit(X_test, y_test)
grid.best_estimator_

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [29]:
grid.best_estimator_.fit(X_test, y_test)

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [30]:
prediction = grid.best_estimator_.predict(X_train)
print_predict_scores(y_train, prediction)

mean_squared_error:
0.12559952038369304

precision_score:
0.6398940864960282

recall_score:
0.1714893952534889

classification_report:
             precision    recall  f1-score   support

          0       0.88      0.98      0.93    242175
          1       0.64      0.17      0.27     38049

avg / total       0.85      0.87      0.84    280224


mean square root: 0.3544002262748897



In [31]:
print('Train score r2:', grid.best_estimator_.score(X_train,y_train))
print('Test score r2:', grid.best_estimator_.score(X_test,y_test))

Train score r2: 0.874400479616
Test score r2: 0.875203408702


In [52]:
print(confusion_matrix(y_train, prediction))

[[238503   3672]
 [ 31524   6525]]


#### Testing Lasso, Ridge, ElasticNet

In [32]:
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error

In [33]:
params = [
    {'alpha': [0.1, 0.2, 0.3, 0.5]}
]
lasso = Lasso()
grid_search_lasso = GridSearchCV(lasso, params, cv=5, scoring='neg_mean_squared_error')
grid_search_lasso.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'alpha': [0.1, 0.2, 0.3, 0.5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [34]:
grid_search_lasso.best_estimator_
lasso_model = grid_search_lasso.best_estimator_
lasso_model.fit(X_train, y_train)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [35]:
prediction2 = lasso_model.predict(X_train)
mse2 = mean_squared_error(prediction2, y_train)
print('mean square root error :\n{}\n'.format(np.sqrt(mse2)))

mean square root error :
0.3425555038725988



In [36]:
print('Train score r2:', lasso_model.score(X_train,y_train))
print('Test score r2:', lasso_model.score(X_test,y_test))

Train score r2: 0.0
Test score r2: -1.72775913421e-06


In [37]:
ridge = Ridge()
grid_seaerch_ridge = GridSearchCV(ridge, params, cv = 3, scoring='neg_mean_squared_error')
grid_seaerch_ridge.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'alpha': [0.1, 0.2, 0.3, 0.5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [38]:
grid_seaerch_ridge.best_score_
ridge_model = grid_seaerch_ridge.best_estimator_
ridge_model.fit(X_train, y_train)

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [39]:
prediction3 = ridge_model.predict(X_train)
mse3 = mean_squared_error(prediction3, y_train)
print('square root:\n{}\n'.format(np.sqrt(mse3)))

square root:
0.31125037835633296



In [40]:
print('Train score r2:', ridge_model.score(X_train,y_train))
print('Test score r2:', ridge_model.score(X_test,y_test))

Train score r2: 0.174422446383
Test score r2: 0.174813348696


In [41]:
en = ElasticNet()
grid_seaerch_en = GridSearchCV(en, params, cv = 3, scoring='neg_mean_squared_error')
grid_seaerch_en.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'alpha': [0.1, 0.2, 0.3, 0.5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [42]:
grid_seaerch_en.best_score_
en_model = grid_seaerch_en.best_estimator_
en_model.fit(X_train, y_train)

ElasticNet(alpha=0.1, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [43]:
prediction4 = en_model.predict(X_train)
mse4 = mean_squared_error(prediction4, y_train)
print('square root:\n{}\n'.format(np.sqrt(mse4)))

square root:
0.3415930880867029



In [44]:
print('Train score r2:', en_model.score(X_train,y_train))
print('Test score r2:', en_model.score(X_test,y_test))

Train score r2: 0.00561114221135
Test score r2: 0.00571305151875


#### KNeighbors

In [45]:
df.columns

Index(['motion', 'light_scaled', 'sound_scaled', 'location_black',
       'location_blue', 'location_green', 'location_orange', 'location_purple',
       'sun_evening', 'sun_morning', 'sun_night', 'sun_noon', 'sun_sunrise',
       'sun_sunset', 'dot_week', 'sound_log_scaled', 'light_log_scaled',
       'daily_light_percentile_25', 'daily_motion_std_dev', 'light_expected'],
      dtype='object')

In [46]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
knn = KNeighborsClassifier(n_neighbors=5)

In [47]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [48]:
prediction5 = knn.predict(X_train)

In [49]:
print_predict_scores(y_train, prediction5)

mean_squared_error:
0.12179542080621217

precision_score:
0.5514885566387261

recall_score:
0.5516045099739809

classification_report:
             precision    recall  f1-score   support

          0       0.93      0.93      0.93    242175
          1       0.55      0.55      0.55     38049

avg / total       0.88      0.88      0.88    280224


mean square root: 0.34899200679415593



In [50]:
print(confusion_matrix(y_train, prediction5))

[[225106  17069]
 [ 17061  20988]]


In [51]:
print('Train score r2:', knn.score(X_train,y_train))
print('Test score r2:', knn.score(X_test,y_test))

Train score r2: 0.878204579194
Test score r2: 0.878158187736


Regression models:
    - linear support regressor
    - linear vs non-linear kernel
    - random forest
    - XG Boost model

Keras - Python library for LSTMs modeling
- PyTorch
